# Wireguard Server on Windows 
[Based on this guide](https://www.henrychang.ca/how-to-setup-wireguard-vpn-server-on-windows/)

In [1]:
%cd ..

C:\Users\Jonny\Human Creative Dropbox\Jonny Hyman\Compute\ResolveCollaboration


In [2]:
from rmc.auth import keys
from pathlib import Path
import subprocess
import os

In [3]:
#!pip install elevate

In [5]:
# Create server config file sans post-up and post-down
s_IP = "8.0.0.1"
s_pk = keys.PrivateKey.generate()
s_Pk = s_pk.public_key()
port = 51820
print(s_pk, s_Pk)
server_config = f"""
[Interface]
# Substitute with the subnet you chose for Wireguard earlier.
Address = {s_IP}/32
# Substitute with your *server's* private key
PrivateKey = {s_pk}
# If you chose a different port earlier when setting up port
# forwarding on your router, update the port here to match.
ListenPort = {port}
# This prevents IPv4 & IPv6 DNS leaks when browsing the web on the VPN
DNS = 1.1.1.1, 8.8.8.8, 2001:4860:4860::8888
"""

aKr3uFswvCtkSjqNIJJ2fKSGZj/l+XzgflBAD29KQ1U= LjCnZDM4HsOkJQebT47jjRyzhx2fFiLLbbj1TnXXpD8=


In [6]:
n=1
c_pk = keys.PrivateKey.generate()
c_Pk = c_pk.public_key()
c_IP = "8.0.0.2"
user = {'name':"Viola_Philharmonic", 'Pk':c_Pk, 'ip':c_IP}

print(c_pk, c_Pk)
server_config += (f"""

# USER {n}: {user['name']}
[Peer]
# Substitute with *this peer*'s public key
PublicKey = {user['Pk']}
# Chose a unique IP within the Wireguard subnet you defined earlier
# that this particular peer will use when connected to the VPN.
AllowedIPs = {user['ip']}/32
                """)

client_config = (f"""
[Interface]
# This MUST match the "AllowedIPs" IP you assigned to this peer in
# the server's config.
Address = {c_IP}/32
# Substitute with *this peer's* private key.
PrivateKey = {c_pk}
# This prevents IPv4 & IPv6 DNS leaks when browsing the web on the VPN
DNS = 1.1.1.1, 8.8.8.8, 2001:4860:4860::8888

[Peer]
# Substitute with your *server's* public key
PublicKey = {s_Pk}
# Your Wireguard server's public IP. If you chose a different port
# earlier when setting up port forwarding on your router, update the
# port here to match.
Endpoint = {s_IP}:{port}
# Informs Wireguard to forward ALL traffic through the VPN.
AllowedIPs = 0.0.0.0/0, ::/0
# If you're be behind a NAT, this will keep the connection alive.
PersistentKeepalive = 25
                """)
print(server_config)
print('----------------')
print(client_config)

iCH04kLvI2nrkbvgbcZfEkAw2fxZzwEIWm7kcypx2U0= 6+yJtB6l2el3e5p8zkR6gDRBH9EY0g2GZvZaE4BYE3U=

[Interface]
# Substitute with the subnet you chose for Wireguard earlier.
Address = 8.0.0.1/32
# Substitute with your *server's* private key
PrivateKey = aKr3uFswvCtkSjqNIJJ2fKSGZj/l+XzgflBAD29KQ1U=
# If you chose a different port earlier when setting up port
# forwarding on your router, update the port here to match.
ListenPort = 51820
# This prevents IPv4 & IPv6 DNS leaks when browsing the web on the VPN
DNS = 1.1.1.1, 8.8.8.8, 2001:4860:4860::8888


# USER 1: Viola_Philharmonic
[Peer]
# Substitute with *this peer*'s public key
PublicKey = 6+yJtB6l2el3e5p8zkR6gDRBH9EY0g2GZvZaE4BYE3U=
# Chose a unique IP within the Wireguard subnet you defined earlier
# that this particular peer will use when connected to the VPN.
AllowedIPs = 8.0.0.2/32
                
----------------

[Interface]
# This MUST match the "AllowedIPs" IP you assigned to this peer in
# the server's config.
Address = 8.0.0.2/32
# 

In [4]:
saveto = Path("C:/Wireguard")
saveto.mkdir(parents=True, exist_ok=True)

with open(saveto / 'Wireguard_server.conf', 'w') as f:
    f.write(server_config)

NameError: name 'server_config' is not defined

In [127]:
# CLIENT
with open('tests/Wireguard_user1.conf', 'w') as f:
    f.write(client_config)

In [17]:
Path("C:/Wireguard/Wireguard_server.conf").exists()

True

```shell
#Start server
C:\Program Files\WireGuard\wireguard.exe /installtunnelservice "C:\wireguard\wg_server.conf"

#Stop server
C:\Program Files\WireGuard\wireguard.exe /uninstalltunnelservice wg_server
```

In [5]:
!wg show

interface: Wireguard_server
  public key: e6bgH5kP8IUtxEi9/LrXy3fW7+1M/8Qo7Plsfg+K+2o=
  private key: (hidden)
  listening port: 51820


In [6]:
def run_return(cmd):
    try:
        out = subprocess.check_output(cmd, shell=True, stderr=subprocess.STDOUT)
        return (str(out,'utf-8')) # empty if successful
    except subprocess.CalledProcessError as e:
        return f"Command {cmd} returned error {e.returncode}"

In [42]:
# assumes we're elevated already
cmd = '''"C:/Program Files/WireGuard/wireguard.exe" /installtunnelservice "C:/Wireguard/Wireguard_server.conf"'''
run_return(cmd) # empty if successful

''

In [9]:
# assumes we're elevated already
cmd = '''"C:/Program Files/WireGuard/wireguard.exe" /uninstalltunnelservice Wireguard_server'''
ret = run_return(cmd) 
ret, type(ret)

('Command "C:/Program Files/WireGuard/wireguard.exe" /uninstalltunnelservice Wireguard_server returned error 1',
 str)

# Set Private profile
```shell
#Open powershell with admin privilege and run the following:

$NetworkProfile = Get-NetConnectionProfile -InterfaceAlias "wg_server"

$NetworkProfile.NetworkCategory = "Private"

Set-NetConnectionProfile -InputObject $NetworkProfile
```

In [11]:
# Get the Interface profile and put it in a variable NetworkProfile
cmd = ('''powershell $NetworkProfile = Get-NetConnectionProfile -InterfaceAlias "Wireguard_server"; '''
        '''$NetworkProfile.NetworkCategory = 'Private'; '''
        '''Set-NetConnectionProfile -InputObject $NetworkProfile''')
print(cmd)
ret = run_return(cmd) # empty if successful
ret

powershell $NetworkProfile = Get-NetConnectionProfile -InterfaceAlias "Wireguard_server"; $NetworkProfile.NetworkCategory = 'Private'; Set-NetConnectionProfile -InputObject $NetworkProfile


'Command powershell $NetworkProfile = Get-NetConnectionProfile -InterfaceAlias "Wireguard_server"; $NetworkProfile.NetworkCategory = \'Private\'; Set-NetConnectionProfile -InputObject $NetworkProfile returned error 1'

In [12]:
?str

Init signature: str(self, /, *args, **kwargs)
Docstring:     
str(object='') -> str
str(bytes_or_buffer[, encoding[, errors]]) -> str

Create a new string object from the given object. If encoding or
errors is specified, then the object must expose a data buffer
that will be decoded using the given encoding and error handler.
Otherwise, returns the result of object.__str__() (if defined)
or repr(object).
encoding defaults to sys.getdefaultencoding().
errors defaults to 'strict'.
Type:           type
Subclasses:     _rstr, LSString, include, SortKey


# Define powershell function to enable sharing
__Save the following to__ `C:\Windows\System32\WindowsPowerShell\v1.0\Modules\wireguard\wireguard.psm1`

```powershell

Function Set-NetConnectionSharing
{
    Param
    (
        [Parameter(Mandatory=$true)]
        [string]
        $LocalConnection,

        [Parameter(Mandatory=$true)]
        [bool]
        $Enabled        
    )

    Begin
    {
        $netShare = $null

        try
        {
            # Create a NetSharingManager object
            $netShare = New-Object -ComObject HNetCfg.HNetShare
        }
        catch
        {
            # Register the HNetCfg library (once)
            regsvr32 /s hnetcfg.dll

            # Create a NetSharingManager object
            $netShare = New-Object -ComObject HNetCfg.HNetShare
        }
    }

    Process
    {
		#Clear Existing Share	       
		$oldConnections = $netShare.EnumEveryConnection |? { $netShare.INetSharingConfigurationForINetConnection.Invoke($_).SharingEnabled -eq $true}           
		foreach($oldShared in $oldConnections)
        {
            Write-Output "... Clearing old connection"
            $oldConfig = $netShare.INetSharingConfigurationForINetConnection.Invoke($oldShared)
            $oldConfig.DisableSharing()                        
        }        
	
        # Find connections
        $InternetConnection = Get-NetRoute | ? DestinationPrefix -eq '0.0.0.0/0' | Get-NetIPInterface | Where ConnectionState -eq 'Connected'        
        $publicConnection = $netShare.EnumEveryConnection |? { $netShare.NetConnectionProps.Invoke($_).Name -eq $InternetConnection.InterfaceAlias }
        $privateConnection = $netShare.EnumEveryConnection |? { $netShare.NetConnectionProps.Invoke($_).Name -eq $LocalConnection }

        # Get sharing configuration
        $publicConfig = $netShare.INetSharingConfigurationForINetConnection.Invoke($publicConnection)
        $privateConfig = $netShare.INetSharingConfigurationForINetConnection.Invoke($privateConnection)

        Write-Output "... Enabling"

        if ($Enabled)
        { 			
            $publicConfig.EnableSharing(0)
            $privateConfig.EnableSharing(1)
        }
        else
        {
            $publicConfig.DisableSharing()
            $privateConfig.DisableSharing()
        }
    }
}
```

In [121]:
Path("C:/Windows/System32/WindowsPowerShell/v1.0/Modules/wireguard/").mkdir(parents=True, exist_ok=True)

with open("C:/Windows/System32/WindowsPowerShell/v1.0/Modules/wireguard/wireguard.psm1", 'w') as f:
    with open("tests/wg_setnetshare.psm1", 'r') as t:
        f.write(t.read())

        
with open("C:/Windows/System32/WindowsPowerShell/v1.0/Modules/wireguard/cleanwmi.vbs", 'w') as f:
    with open("tests/wg_cleanwmientries.vbs", 'r') as t:
        f.write(t.read())

#print(function)

```powershell
# another powershell with admin privilege
#"wg_server" is the wireguard adapter name, replace it if you have something different.
#Enable NAT
Set-NetConnectionSharing "wg_server" $true

#Disable NAT
Set-NetConnectionSharing "wg_server" $false
```

In [135]:
#Enable NAT
cmd = '''powershell Set-NetConnectionSharing "Wireguard_server" $true'''
print(run_return(cmd))

... Clearing old connection
... Clearing old connection
>>> WMI Cleanup
DONE

SharingEnabled SharingConnectionType InternetFirewallEnabled
-------------- --------------------- -----------------------
          True                     0                    True
          True                     1                    True





In [136]:
#Disable NAT
cmd = '''powershell Set-NetConnectionSharing "Wireguard_server" $false'''
print(run_return(cmd))

... Clearing old connection
... Clearing old connection
>>> WMI Cleanup
DONE

SharingEnabled SharingConnectionType InternetFirewallEnabled
-------------- --------------------- -----------------------
         False                     0                    True
         False                     0                    True





# Making the class

In [16]:
proc = Popen(['wg','show'], stdout=PIPE, stderr=PIPE)
wg, err = proc.communicate()
wg = str(wg,'utf-8')
print(wg)

interface: Wireguard_rmcs
  public key: IkVgml1bWhu9X1c4xxrPuh2/MMIYaKcerRVOtghEtD4=
  private key: (hidden)
  listening port: 51820



In [14]:
from subprocess import Popen, PIPE

def peers():
    """ Returns a dict containing all peers in the Server config.
            like: {Pk: {'ip':'...', 'handshake': True}}
    """

    proc = Popen(['wg','show'], stdout=PIPE, stderr=PIPE)
    wg, err = proc.communicate()
    wg = str(wg,'utf-8')

    peers = {}

    if "peer: " in wg:
        peers_conf = wg.split("peer: ")

        # [1:] because interface/server is 0
        for peer in peers_conf[1:]:

            Pk = peer.split('\n')[0].strip()
            handshake = False

            for row in peer.split('\n'):
                if 'allowed ips' in row:
                    ip = row.strip().split(':')[1].strip()

                if 'handshake' in row:
                    handshake = True

            peers[Pk] = {"ip": ip, 'handshake':handshake}

    return peers

peers()

{}

In [134]:
from subprocess import Popen, PIPE

from pathlib import Path
import subprocess
import os

from ipaddress import ip_address, ip_network
from rmc.auth import keys

class WireguardServer_Windows:
    """ Performs all setup for a Wireguard Windows 10 server. Must be run as Admin """

    def __init__(self, force_reset=False, config=None,
                        port=51820, subnet = "9.0.0.0/24"):

        if config is not None:
            self.subnet = config['auth']['subnet']
            self.port = config['wireguard']['port']
            self.pk = config['wireguard']['pk']
            self.Pk = config['wireguard']['Pk']
            return

        self.subnet = subnet
        self.port = port

        # -------- Setup :
        
        Path("C:/Wireguard").mkdir(parents=True, exist_ok=True)
        Path("C:/Windows/System32/WindowsPowerShell/v1.0/Modules/wireguard/").mkdir(parents=True, exist_ok=True)
        
        # install function to give the wireguard tunnel internet access
        with open("C:/Windows/System32/WindowsPowerShell/v1.0/Modules/wireguard/wireguard.psm1", 'w') as psm1:
            psm1.write(enablesharing_powershell)

        # install VBS script to clean up WMI sharing instances
        with open("C:/Windows/System32/WindowsPowerShell/v1.0/Modules/wireguard/cleanwmi.vbs", 'w') as vbs:
            vbs.write(cleanwmi_vbs)

        # -------- Configure

        self.pk, self.Pk = self.pk_Pk_pair()

    def run_return(self, cmd):
        """ Run a command, capture output, don't crash"""
        try:
            out = subprocess.check_output(cmd, shell=True, stderr=subprocess.STDOUT)
            return (str(out,'utf-8')) # empty if successful
        except subprocess.CalledProcessError as e:
            return f"Command {cmd} returned error {e.returncode}"
        
    def update_config(self, userlist):
        ''' Overwrite the config with the current userlist'''

        first_ip = next(ip_network(self.subnet).hosts())

        # Interface always first
        server_config = (f"""
[Interface]
# Substitute with the subnet you chose for Wireguard earlier.
Address = {first_ip}/32
# Substitute with your *server's* private key
PrivateKey = {self.pk}
# If you chose a different port earlier when setting up port
# forwarding on your router, update the port here to match.
ListenPort = {self.port}
# This prevents IPv4 & IPv6 DNS leaks when browsing the web on the VPN
DNS = 1.1.1.1, 8.8.8.8, 2001:4860:4860::8888
        """)

        print(f">>> Added Server to config")

        # Peers

        for n, user in enumerate(userlist[1:]):

            # Authentication occured?
            if user['Pk'] != "":

                print(f">>> Added {user['name']}, {user['Pk']}, {user['ip']} to config")

                server_config += (f"""

# USER {n}: {user['name']}
[Peer]
# Substitute with *this peer*'s public key
PublicKey = {user['Pk']}
# Chose a unique IP within the Wireguard subnet you defined earlier
# that this particular peer will use when connected to the VPN.
AllowedIPs = {user['ip']}/32
                """)

        if self.state:
            self.down()
            do_up = True
        else:
            do_up = False

        with open("C:/Wireguard/Wireguard_server.conf", 'w') as conf:
            conf.write(server_config)

        if do_up:
            self.up()

    @property
    def state(self):
        return self.peers() != {}

    def up(self):

        # already up?
        if self.state:
            # Don't go up if already up
            print('... Wireguard is already up')
            return
        else:
            
            cmd = '''"C:/Program Files/WireGuard/wireguard.exe" /installtunnelservice "C:/Wireguard/Wireguard_server.conf"'''
            up = self.run_return(cmd)
            print("... Installed configuration", str(up,'utf-8')=='')

            # Set the network profile to be in Private category
            cmd = ('''powershell $NetworkProfile = Get-NetConnectionProfile -InterfaceAlias "Wireguard_server"; '''
                    '''$NetworkProfile.NetworkCategory = 'Private'; '''
                    '''Set-NetConnectionProfile -InputObject $NetworkProfile''')
            
            net = self.run_return(cmd) # empty if successful
            print("... Set profile to Private", str(net,'utf-8')=='')
            
            #Enable NAT
            cmd = '''powershell Set-NetConnectionSharing "Wireguard_server" $true'''
            nat = self.run_return(cmd) # text DONE if successful
            print("... Enabled connection sharing", "DONE" in str(nat,'utf-8'))

    def show(self):
        show = subprocess.check_output("wg show", shell=True, stderr=subprocess.STDOUT)
        print(str(output,'utf-8')) # returns empty if

    def down(self):
        if not self.state:
            print('... Wireguard is already down')
            return

        # Disable NAT
        cmd = '''powershell Set-NetConnectionSharing "Wireguard_server" $false'''
        nat = self.run_return(cmd) # empty if successful
        print("... Disabled connection sharing", "DONE" in str(nat,'utf-8'))
        
        # Uninstall tunnel service
        cmd = '''"C:/Program Files/WireGuard/wireguard.exe" /uninstalltunnelservice Wireguard_server'''
        down = run_return(cmd)
        print("... Uninstalled configuration", str(down,'utf-8')=='')

    def pk_Pk_pair(self):
        pk = keys.PrivateKey.generate()
        Pk = pk.public_key()

        return pk, Pk

    def peers(self):
        """ Returns a dict containing all peers in the Server config.
                like: {Pk: {'ip':'...', 'handshake': True}}
        """

        proc = Popen(['wg','show'], stdout=PIPE, stderr=PIPE)
        wg, err = proc.communicate()
        wg = str(wg,'utf-8')

        peers = {}

        if "peer: " in wg:
            peers_conf = wg.split("peer: ")

            # [1:] because interface/server is 0
            for peer in peers_conf[1:]:

                Pk = peer.split('\n')[0].strip()
                handshake = False

                for row in peer.split('\n'):
                    if 'allowed ips' in row:
                        ip = row.strip().split(':')[1].strip()

                    if 'handshake' in row:
                        handshake = True

                peers[Pk] = {"ip": ip, 'handshake':handshake}

        return peers

cleanwmi_vbs = """'Clean up WMI Entries so we can make a new one
'https://github.com/billchaison/Windows-Trix
Wscript.Echo ">>> WMI Cleanup"

set WMI = GetObject("WinMgmts:\root\Microsoft\HomeNet")
set objs1 = WMI.ExecQuery("SELECT * FROM HNet_ConnectionProperties WHERE IsIcsPrivate = TRUE")
for each obj in objs1
   obj.IsIcsPrivate = FALSE
   obj.Put_
next
set objs2 = WMI.ExecQuery("SELECT * FROM HNet_ConnectionProperties WHERE IsIcsPublic = TRUE")
for each obj in objs2
   obj.IsIcsPublic = FALSE
   obj.Put_
next
"""
    
enablesharing_powershell = """Function Set-NetConnectionSharing
{
    Param
    (
        [Parameter(Mandatory=$true)]
        [string]
        $LocalConnection,

        [Parameter(Mandatory=$true)]
        [bool]
        $Enabled        
    )

    Begin
    {
        $netShare = $null

        try
        {
            # Create a NetSharingManager object
            $netShare = New-Object -ComObject HNetCfg.HNetShare
        }
        catch
        {
            # Register the HNetCfg library (once)
            regsvr32 /s hnetcfg.dll

            # Create a NetSharingManager object
            $netShare = New-Object -ComObject HNetCfg.HNetShare
        }
    }

    Process
    {
		#Clear Existing Share	       
		$oldConnections = $netShare.EnumEveryConnection |? { 
            $netShare.INetSharingConfigurationForINetConnection.Invoke($_).SharingEnabled -eq $true
        }           
        
		foreach($oldShared in $oldConnections)
        {
            Write-Output "... Clearing old connection"
            $oldConfig = $netShare.INetSharingConfigurationForINetConnection.Invoke($oldShared)
            $oldConfig.DisableSharing()                        
        }        
        
        # Clean up WMI
        # https://github.com/billchaison/Windows-Trix
        & cscript /nologo "C:/Windows/System32/WindowsPowerShell/v1.0/Modules/wireguard/cleanwmi.vbs"

        # Find connections
        $InternetConnection = Get-NetRoute | ? DestinationPrefix -eq '0.0.0.0/0' | Get-NetIPInterface | Where ConnectionState -eq 'Connected'        
        $publicConnection = $netShare.EnumEveryConnection |? { $netShare.NetConnectionProps.Invoke($_).Name -eq $InternetConnection.InterfaceAlias }
        $privateConnection = $netShare.EnumEveryConnection |? { $netShare.NetConnectionProps.Invoke($_).Name -eq $LocalConnection }

        # Get sharing configuration
        $publicConfig = $netShare.INetSharingConfigurationForINetConnection.Invoke($publicConnection)
        $privateConfig = $netShare.INetSharingConfigurationForINetConnection.Invoke($privateConnection)

        if ($Enabled)
        { 			
            # Enable sharing (0 - public, 1 - private)
            $publicConfig.EnableSharing(0)
            $privateConfig.EnableSharing(1)
        }
        else
        {
            $publicConfig.DisableSharing()
            $privateConfig.DisableSharing()
        }
        
        Write-Output ">>> DONE"
        
        $publicConfig = $netShare.INetSharingConfigurationForINetConnection.Invoke($publicConnection)
        $privateConfig = $netShare.INetSharingConfigurationForINetConnection.Invoke($privateConnection)
        
        Write-Output $publicConfig
        Write-Output $privateConfig
    }
}
"""

In [140]:
?subprocess.check_output

Signature: subprocess.check_output(*popenargs, timeout=None, **kwargs)
Docstring:
Run command with arguments and return its output.

If the exit code was non-zero it raises a CalledProcessError.  The
CalledProcessError object will have the return code in the returncode
attribute and output in the output attribute.

The arguments are the same as for the Popen constructor.  Example:

>>> check_output(["ls", "-l", "/dev/null"])
b'crw-rw-rw- 1 root root 1, 3 Oct 18  2007 /dev/null\n'

The stdout argument is not allowed as it is used internally.
To capture standard error in the result, use stderr=STDOUT.

>>> check_output(["/bin/sh", "-c",
...               "ls -l non_existent_file ; exit 0"],
...              stderr=STDOUT)
b'ls: non_existent_file: No such file or directory\n'

There is an additional optional argument, "input", allowing you to
pass a string to the subprocess's stdin.  If you use this argument
you may not also use the Popen constructor's "stdin" argument, as
it too will be 